In [1]:
# math and robotics
import pathlib
import numpy as np
import spatialmath as sm
import quaternion
from comau_msgs.srv import SetIO, SetIORequest


# ROS
import actionlib
import rospy
import hrr_common
from hr_recycler_msgs.msg import (
    AdaptiveGraspingAction, AdaptiveGraspingGoal,
    CuttingAction, CuttingGoal,
    ChangeToolAction, ChangeToolGoal,    
    FingerGraspAction, FingerGraspGoal,    
    PushOpenAction, PushOpenGoal,
    VacuumGraspingAction, VacuumGraspingGoal,
    UnscrewAction, UnscrewGoal
)
from hrr_msgs.msg import CalibrateCobotAction, CalibrateCobotGoal

# set printing and plotting options
np.set_printoptions(precision=10, suppress=True)

from hr_recycler_msgs.msg import CuttingAction, CuttingFeedback, CuttingResult, CuttingGoal, SkillResult, CobotState, MaterialType
import actionlib
from hr_recycler_msgs.msg import ToolType

from hr_recycler_msgs.msg import PlannerAction , PlannerGoal
import hrr_common as hrr_cm

In [2]:
rospy.init_node('full_tool_change_routine_demo')

In [3]:
cobot_ns = hrr_common.ros_utils.fix_prefix(hrr_common.ros_utils.get_param("/cobot_ns"))
change_tool_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}change_tool_action_srv_name")
calibration_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}calibration_action_srv_name")
cutting_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}cutting_action_srv_name")
grinding_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}grinding_action_srv_name")
pc_opening_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}pc_opening_action_srv_name")
sensitive_grasping_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}sensitive_grasping_action_srv_name")
unscrew_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}unscrew_action_srv_name")
vacuum_pick_place_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}vacuum_pick_place_action_srv_name")
finger_grasping_action_srv_name= hrr_common.ros_utils.get_param(f"{cobot_ns}finger_grasping_action_srv_name")
#pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)

def get_client(action_topic, ActionClass, timeout_secs=2.0):
    client = actionlib.SimpleActionClient(action_topic, ActionClass)
    client.wait_for_server(timeout=rospy.Duration(timeout_secs))
    return client
def get_result(cl):
    cl.wait_for_result()
    return cl.get_result()

def cancel(cl):
    cl.cancel_all_goals()
    
    
def set_cutting_goal(pointA, pointB):
    material= MaterialType()
    material = material.PLASTIC

    normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalB")
    normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalA")

    goal=CuttingGoal()
    goal.surface_normal_start=normalA
    goal.surface_normal_end=normalB
    goal.start_location=pointA
    goal.end_location=pointB
    goal.material.material_type=material
    goal.timeout=5000
    goal.thickness = 0.005 #not important. can be anything
    return goal
    
calibrate_client = get_client(calibration_action_srv_name, CalibrateCobotAction)   
vacuum_client = get_client(vacuum_pick_place_action_srv_name, VacuumGraspingAction)
pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)
tool_change_client = get_client(change_tool_action_srv_name, ChangeToolAction)
shaftgrinder_client=get_client(cutting_action_srv_name, CuttingAction)
grasping_client = get_client(sensitive_grasping_action_srv_name, AdaptiveGraspingAction)

In [4]:
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")
#cobot.change_tool("nothing")

[rosout] [/full_tool_change_routine_demo] Initialize MATLAB-Motion planner @ /home/hrr_cobot/_ros/hr_recycler_ws/src/planner/src
[rosout] [/full_tool_change_routine_demo] connected to Matlab
Shutting down global node /matlab_global_node_34791 with NodeURI http://192.168.2.87:36687/
The value of the ROS_IP environment variable, 192.168.2.87, will be used to set the advertised address for the ROS node.
Initializing global node /matlab_global_node_73730 with NodeURI http://192.168.2.87:43159/
[rosout] [/full_tool_change_routine_demo] MATLAB-Initialization done


pybullet build time: May  8 2021 05:48:13


[rosout] some functions are not yet compiled. Expect delays upon first call


In [12]:
cobot.open_tool_changer()

In [6]:
cobot.change_tool("nothing")

True

### Screwdriver

In [11]:
tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.SCREW_DRIVER
tool_change_client.send_goal(tc_goal)

### Shaftgrinder

In [10]:
tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.SHAFT_GRINDER
tool_change_client.send_goal(tc_goal)

### Finger_vacuum

In [7]:
tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.WSG_50
tool_change_client.send_goal(tc_goal)
# tool_change_client.cancel_all_goals()

### Adaptive_gripper

In [9]:
tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.WSG_50_DSA
tool_change_client.send_goal(tc_goal)

### vacuum

In [8]:
tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.VACUUM_GRIPPER
tool_change_client.send_goal(tc_goal)

In [ ]:
tool_change_client.cancel_all_goals()

In [ ]:
cobot.tool